### 6. Aim: Demonstrate and Analyse the results sets obtained from Bayesian belief network principle.

In [1]:
import sys
from urllib.request import urlopen
import numpy as np
import pandas as pd

In [2]:
heartDisease_df = pd.read_csv('heart.csv')


In [3]:
heartDisease_df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [4]:
heartDisease_df.drop(['ca', 'slope', 'thal', 'oldpeak'], axis=1, inplace=True)

In [5]:
heartDisease_df.replace('?', np.nan, inplace=True)

In [6]:
#conda install -c ankurankan pgmpy

In [7]:
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
model = BayesianNetwork([('age', 'trestbps'),
('age', 'fbs'), ('sex', 'trestbps'),
('exang', 'trestbps'), ('trestbps', 'target'),
('fbs', 'target'), ('target', 'restecg'),
('target', 'thalach'), ('target', 'chol')])
# Learing CPDs using Maximum Likelihood Estimators
model.fit(heartDisease_df, estimator=MaximumLikelihoodEstimator)

In [8]:
print(model.get_cpds('age'))
print(model.get_cpds('chol'))
print(model.get_cpds('sex'))
model.get_independencies()


+---------+------------+
| age(29) | 0.00390244 |
+---------+------------+
| age(34) | 0.00585366 |
+---------+------------+
| age(35) | 0.0146341  |
+---------+------------+
| age(37) | 0.00585366 |
+---------+------------+
| age(38) | 0.0117073  |
+---------+------------+
| age(39) | 0.0136585  |
+---------+------------+
| age(40) | 0.0107317  |
+---------+------------+
| age(41) | 0.0312195  |
+---------+------------+
| age(42) | 0.0253659  |
+---------+------------+
| age(43) | 0.0253659  |
+---------+------------+
| age(44) | 0.035122   |
+---------+------------+
| age(45) | 0.0243902  |
+---------+------------+
| age(46) | 0.022439   |
+---------+------------+
| age(47) | 0.017561   |
+---------+------------+
| age(48) | 0.022439   |
+---------+------------+
| age(49) | 0.0165854  |
+---------+------------+
| age(50) | 0.0204878  |
+---------+------------+
| age(51) | 0.0380488  |
+---------+------------+
| age(52) | 0.0419512  |
+---------+------------+
| age(53) | 0.0253659  |


(sex ⟂ age, fbs, exang)
(sex ⟂ age, fbs | exang)
(sex ⟂ fbs, exang | age)
(sex ⟂ age, exang | fbs)
(sex ⟂ thalach, restecg, chol | target)
(sex ⟂ fbs | age, exang)
(sex ⟂ age | fbs, exang)
(sex ⟂ thalach, restecg, chol | exang, target)
(sex ⟂ thalach, chol | restecg, target)
(sex ⟂ thalach, fbs, restecg, chol, target | age, trestbps)
(sex ⟂ thalach, restecg, chol, target | fbs, trestbps)
(sex ⟂ thalach, restecg, chol | target, trestbps)
(sex ⟂ exang | age, fbs)
(sex ⟂ thalach, restecg, chol | age, target)
(sex ⟂ thalach, restecg, chol | fbs, target)
(sex ⟂ restecg, chol | thalach, target)
(sex ⟂ thalach, restecg | chol, target)
(sex ⟂ thalach, chol | restecg, exang, target)
(sex ⟂ fbs, thalach, restecg, chol, target | age, exang, trestbps)
(sex ⟂ thalach, restecg, chol, target | fbs, exang, trestbps)
(sex ⟂ thalach, restecg, chol | target, exang, trestbps)
(sex ⟂ thalach, restecg, chol | age, exang, target)
(sex ⟂ thalach, restecg, chol | fbs, exang, target)
(sex ⟂ restecg, chol | thal

In [9]:
from pgmpy.inference import VariableElimination
HeartDisease_infer = VariableElimination(model)

In [10]:
q = HeartDisease_infer.query(variables=['target'], evidence={'age': 42},joint=False)
print(q['target'])


+-----------+---------------+
| target    |   phi(target) |
+===========+===============+
| target(0) |        0.4426 |
+-----------+---------------+
| target(1) |        0.5574 |
+-----------+---------------+


In [11]:
q = HeartDisease_infer.query(variables=['target'],evidence={'chol': 318},joint=False)
print(q['target'])

+-----------+---------------+
| target    |   phi(target) |
+===========+===============+
| target(0) |        0.5625 |
+-----------+---------------+
| target(1) |        0.4375 |
+-----------+---------------+
